# Import & Ingest

In [30]:
import sys
import pathlib
SOURCE_PATH = pathlib.Path.cwd().resolve().parent
sys.path.append(str(SOURCE_PATH))

In [31]:
from config import *
from utils import ingest_kaggle_database
import pandas as pd

In [32]:
ingest_kaggle_database(SOCCER_DATABASE)

In [33]:
%load_ext sql
%sql sqlite:///../data/database.sqlite
%config SqlMagic.style = "_deprecated_default"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [34]:
%%sql
SELECT name FROM sqlite_schema
WHERE type = "table"
AND name NOT LIKE "sqlite%"
ORDER BY 1;

 * sqlite:///../data/database.sqlite
Done.


name
Country
League
Match
Player
Player_Attributes
Team
Team_Attributes


# Introductory Analysis & Validation Checks (Aiden)

In [35]:
%%sql
-- Count of matches in Match table
SELECT COUNT(*) FROM Match;

 * sqlite:///../data/database.sqlite
Done.


COUNT(*)
25979


In [36]:
%%sql
-- Count of unique teams
SELECT COUNT(DISTINCT team_long_name) AS unique_team_count
FROM Team;

 * sqlite:///../data/database.sqlite
Done.


unique_team_count
296


In [37]:
%%sql
-- Displays sample data of Team table
SELECT * FROM Team LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC Genk,GEN
2,9993,675,Beerschot AC,BAC
3,10000,15005,SV Zulte-Waregem,ZUL
4,9994,2007,Sporting Lokeren,LOK


In [38]:
%%sql
-- Displays sample data of Team_Attributes
SELECT * FROM Team_Attributes LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
1,434,9930,2010-02-22 00:00:00,60,Balanced,None,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
2,434,9930,2014-09-19 00:00:00,52,Balanced,48,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,434,9930,2015-09-10 00:00:00,47,Balanced,41,Normal,54,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
4,77,8485,2010-02-22 00:00:00,70,Fast,None,Little,70,Long,Organised,70,Risky,70,Lots,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover


In [39]:
%%sql
-- Displays sample data of Player_Attributes
SELECT * FROM Player_Attributes LIMIT 4;

 * sqlite:///../data/database.sqlite
Done.


id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,44,71,61,44,51,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45,54,48,65,69,69,6,11,10,8,8
2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,44,71,61,44,51,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45,54,48,65,69,69,6,11,10,8,8
3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,44,71,61,44,51,45,39,64,49,60,64,59,47,65,55,58,54,76,35,63,41,45,54,48,65,66,69,6,11,10,8,8
4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,43,70,60,43,50,44,38,63,48,60,64,59,46,65,54,58,54,76,34,62,40,44,53,47,62,63,66,5,10,9,7,7


In [40]:
%%sql
-- Top 25 rated players
SELECT overall_rating
FROM Player_Attributes
ORDER BY overall_rating DESC
LIMIT 25;

 * sqlite:///../data/database.sqlite
Done.


overall_rating
94
94
94
94
94
94
94
94
94
94


In [41]:
%%sql
-- Count of unique players
SELECT COUNT(DISTINCT player_fifa_api_id) AS unique_players
FROM Player_Attributes;

 * sqlite:///../data/database.sqlite
Done.


unique_players
11062


# Exploratory Analysis & Visualization (Keano)

In [42]:
%%sql
-- Build Team Strength Ratings (Offense + Defense):
-- This compresses 40+ player stats into meaningful team metrics.
CREATE VIEW Team_Player_Strength AS
SELECT
m.match_api_id,
AVG(p.overall_rating) AS avg_rating,
AVG(p.finishing + p.shot_power + p.positioning) AS attack_strength,
AVG(p.interceptions + p.marking + p.standing_tackle + p.sliding_tackle) AS defense_strength,
AVG(p.gk_reflexes + p.gk_handling + p.gk_positioning) AS gk_strength
FROM Match m
JOIN Player_Attributes p
ON p.player_api_id IN (
m.home_player_1, m.home_player_2, m.home_player_3, m.home_player_4,
m.home_player_5, m.home_player_6, m.home_player_7, m.home_player_8,
m.home_player_9, m.home_player_10, m.home_player_11,
m.away_player_1, m.away_player_2, m.away_player_3, m.away_player_4,
m.away_player_5, m.away_player_6, m.away_player_7, m.away_player_8,
m.away_player_9, m.away_player_10, m.away_player_11
)
GROUP BY m.match_api_id;

 * sqlite:///../data/database.sqlite
(sqlite3.DatabaseError) malformed database schema (Team_Player_Strength) - view Team_Player_Strength already exists
[SQL: -- Build Team Strength Ratings (Offense + Defense):
-- This compresses 40+ player stats into meaningful team metrics.
CREATE VIEW Team_Player_Strength AS
SELECT
m.match_api_id,
AVG(p.overall_rating) AS avg_rating,
AVG(p.finishing + p.shot_power + p.positioning) AS attack_strength,
AVG(p.interceptions + p.marking + p.standing_tackle + p.sliding_tackle) AS defense_strength,
AVG(p.gk_reflexes + p.gk_handling + p.gk_positioning) AS gk_strength
FROM Match m
JOIN Player_Attributes p
ON p.player_api_id IN (
m.home_player_1, m.home_player_2, m.home_player_3, m.home_player_4,
m.home_player_5, m.home_player_6, m.home_player_7, m.home_player_8,
m.home_player_9, m.home_player_10, m.home_player_11,
m.away_player_1, m.away_player_2, m.away_player_3, m.away_player_4,
m.away_player_5, m.away_player_6, m.away_player_7, m.away_player_8,
m.away_pla

In [43]:
%%sql
SELECT * FROM Team_Player_Strength LIMIT 15;

 * sqlite:///../data/database.sqlite
(sqlite3.OperationalError) no such table: Team_Player_Strength
[SQL: SELECT * FROM Team_Player_Strength LIMIT 15;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [44]:
%%sql
-- Best finishers:
SELECT team_api_id, shooting_quality
FROM Shot_Conversion_Power
ORDER BY shooting_quality DESC;

 * sqlite:///../data/database.sqlite
(sqlite3.OperationalError) no such table: Shot_Conversion_Power
[SQL: -- Best finishers:
SELECT team_api_id, shooting_quality
FROM Shot_Conversion_Power
ORDER BY shooting_quality DESC;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [45]:
%%sql
-- Defensive kings:
SELECT team_api_id, defense_resilience
FROM Defensive_Resilience
ORDER BY defense_resilience DESC
LIMIT 10;

 * sqlite:///../data/database.sqlite
(sqlite3.OperationalError) no such table: Defensive_Resilience
[SQL: -- Defensive kings:
SELECT team_api_id, defense_resilience
FROM Defensive_Resilience
ORDER BY defense_resilience DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [46]:
%%sql
-- Top attacking teams:
SELECT team_api_id, attack_efficiency
FROM Attacking_Efficiency
ORDER BY attack_efficiency DESC
LIMIT 10;

 * sqlite:///../data/database.sqlite
(sqlite3.OperationalError) no such table: Attacking_Efficiency
[SQL: -- Top attacking teams:
SELECT team_api_id, attack_efficiency
FROM Attacking_Efficiency
ORDER BY attack_efficiency DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [47]:
%%sql
-- Tactical Profile and Spider Chart:
SELECT
t.team_long_name,
AVG(ta.buildUpPlaySpeed) AS build_speed,
AVG(ta.chanceCreationShooting) AS shooting,
AVG(ta.chanceCreationPassing) AS passing,
AVG(ta.chanceCreationCrossing) AS crossing,
AVG(ta.defencePressure) AS pressure,
AVG(ta.defenceAggression) AS aggression
FROM Team_Attributes ta
JOIN Team t ON t.team_api_id = ta.team_api_id
WHERE t.team_long_name = 'FC Barcelona'
GROUP BY t.team_long_name;

 * sqlite:///../data/database.sqlite
Done.


team_long_name,build_speed,shooting,passing,crossing,pressure,aggression
FC Barcelona,35.833333333333336,53.0,45.166666666666664,33.333333333333336,64.33333333333333,54.5


In [48]:
%%sql
SELECT
home_team_api_id AS team_id,
COUNT(*) AS matches,
SUM(home_team_goal > away_team_goal) AS wins,
ROUND(100.0 * SUM(home_team_goal > away_team_goal) / COUNT(*), 2) AS win_rate
FROM Match
GROUP BY home_team_api_id
ORDER BY win_rate DESC
LIMIT 15;

 * sqlite:///../data/database.sqlite
Done.


team_id,matches,wins,win_rate
8634,152,131,86.18
8633,152,129,84.87
9773,124,102,82.26
9772,124,102,82.26
9823,136,109,80.15
9925,152,120,78.95
8640,136,105,77.21
10260,152,116,76.32
8593,136,103,75.74
8635,106,79,74.53


In [49]:
%%sql
-- All Team Win/Losses:
SELECT
t.team_long_name,
SUM(CASE
WHEN m.home_team_api_id = t.team_api_id AND m.home_team_goal > m.away_team_goal THEN 1
WHEN m.away_team_api_id = t.team_api_id AND m.away_team_goal > m.home_team_goal THEN 1
ELSE 0
END) AS wins,
SUM(CASE
WHEN m.home_team_api_id = t.team_api_id AND m.home_team_goal < m.away_team_goal THEN 1
WHEN m.away_team_api_id = t.team_api_id AND m.away_team_goal < m.home_team_goal THEN 1
ELSE 0
END) AS losses
FROM Team t
LEFT JOIN Match m ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
GROUP BY t.team_long_name
ORDER BY wins DESC;

 * sqlite:///../data/database.sqlite
(sqlite3.DatabaseError) database disk image is malformed
[SQL: -- All Team Win/Losses:
SELECT
t.team_long_name,
SUM(CASE
WHEN m.home_team_api_id = t.team_api_id AND m.home_team_goal > m.away_team_goal THEN 1
WHEN m.away_team_api_id = t.team_api_id AND m.away_team_goal > m.home_team_goal THEN 1
ELSE 0
END) AS wins,
SUM(CASE
WHEN m.home_team_api_id = t.team_api_id AND m.home_team_goal < m.away_team_goal THEN 1
WHEN m.away_team_api_id = t.team_api_id AND m.away_team_goal < m.home_team_goal THEN 1
ELSE 0
END) AS losses
FROM Team t
LEFT JOIN Match m ON t.team_api_id = m.home_team_api_id OR t.team_api_id = m.away_team_api_id
GROUP BY t.team_long_name
ORDER BY wins DESC;]
(Background on this error at: https://sqlalche.me/e/20/4xp6)
